In [2]:
%pip install -q openai
%pip install -q langchain langchain-openai
%pip install -q faiss-cpu
%pip install -q langchain-community
%pip install -q tiktoken
%pip install -q openai langchain pypdf chroma
%pip install -q unstructured unstructured-inference
%pip install -q pymupdf
%pip install -q sentence-transformers
%pip install -q fuzzywuzzy
%pip install --upgrade --quiet langchain langchain-openai langchainhub
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.1 requires langchain-core<0.4,>=0.3, but you have langchain-core 0.2.41 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.12 requires langchain<0.3.0,>=0.2.13, but you have langchain 0.3.2 which is incompatible.
langchain-community 0.2.12 requires langchain-core<0.3.0,>=0.2.30, but you have langchain-core 0.3.8 which is incompatible.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.8 which is incompatible.
langchain-mongodb 0.1.8 requires langchain-core<0.3.0,>=0.2.21, but you have langchain-core 0.3.8 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.document_loaders import CSVLoader
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferWindowMemory
import pandas as pd
from langchain.vectorstores import VectorStore

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [5]:
# 방법 1: raw 문자열 사용
path = r'E:\0930\dog_data_toal.csv'

# 또는 방법 2: 슬래시 사용
path = 'E:/0930/dog_data_toal.csv'

df = pd.read_csv(path, encoding='utf-8-sig')

df.head()

,URL,main,sub,info,category,trainer,type,breed,Unnamed: 8
0,https://www.youtube.com/watch?v=yCqzo_DIbJo&li...,반려동물의 식사 습관,반려동물의 식사 습관은 건강에 매우 중요합니다.\n아기 강아지는 밥을 빨리 먹어야 ...,반려동물에게 밥을 주었지만 먹지 않는 경우가 있습니다.\n특히 아기 강아지들은 밥을...,정보,강형욱,강아지,공통,
1,https://www.youtube.com/watch?v=5PF5dMzSLZE&li...,강아지 훈련의 기초,강아지 훈련에서 가슴줄의 사용은 필수적입니다.\n훈련 초기에는 가벼운 목줄부터 시작...,강아지 훈련에서 가슴줄을 사용하는 것이 중요합니다.\n처음에는 가벼운 리본이나 목줄...,기본교육,강형욱,강아지,공통,NaN
2,https://www.youtube.com/watch?v=VSzERyOQ8sw&li...,강아지 훈련 방법,이 영상에서는 강아지 훈련 방법에 대해 설명합니다.\n강아지와의 상호작용을 통해 간...,강아지와 같은 방향으로 서서 간식을 주는 방법을 설명합니다.\n왼쪽에서 간식을 주며...,기본교육,강형욱,강아지,공통,NaN
3,https://www.youtube.com/watch?v=2hwUIL4l_FA&li...,기본적인 훈련 방법,"강아지 훈련의 기본적인 방법을 소개합니다.\n손을 위로 올리는 동작부터 시작하여, ...",기본적인 것부터 해보겠습니다.\n손을 위로 올려 보세요.\n일부 친구들은 낯선 사람...,기본교육,강형욱,강아지,공통,NaN
4,https://www.youtube.com/watch?v=2hwUIL4l_FA&li...,훈련의 진행 과정,"훈련의 진행 과정에서 손을 많이 사용해야 하며, 강아지를 서게 한 후 엎드리게 하는...",훈련을 진행할 때는 손을 많이 써야 합니다.\n엎드리기를 시키기 전에 먼저 서게 한...,기본교육,강형욱,강아지,공통,NaN


In [6]:
import pandas as pd
import json
from sentence_transformers import SentenceTransformer
import numpy as np
# JSON 구조로 변환하는 함수
def create_json_structure(row):
    # 'main'과 'sub', 'info' 컬럼의 텍스트를 결합
    combined_text = f"{row['main']} {' '.join(row['sub'])} {' '.join(row['info'])}"
        
    return {
        "id": row.name,  # pandas DataFrame의 인덱스를 ID로 사용
        "url": row['URL'],
        "main": row['main'],
        "sub": row['sub'].split('\n'),  # '\n'으로 분리하여 리스트로 저장
        "info": row['info'].split('\n'),  # '\n'으로 분리하여 리스트로 저장
        "category": row['category'],
        "trainer": row['trainer'],
        "type": row['type'],
        "breed": row['breed'],
    }

# 각 행을 JSON 구조로 변환
json_data = [create_json_structure(row) for _, row in df.iterrows()]

# JSON 파일로 저장
with open('dog_training_data.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print("JSON 파일이 생성되었습니다.")

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


JSON 파일이 생성되었습니다.


In [7]:
json_data

[{'id': 0,
  'url': 'https://www.youtube.com/watch?v=yCqzo_DIbJo&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=23&pp=iAQB',
  'main': '반려동물의 식사 습관',
  'sub': ['반려동물의 식사 습관은 건강에 매우 중요합니다.',
   '아기 강아지는 밥을 빨리 먹어야 하며, 이를 통해 기력을 유지할 수 있습니다.',
   '하루에 두 끼 또는 네 끼를 주는 것이 적절하며, 간식과 밥을 동일하게 생각해야 합니다.',
   '반려동물의 식사량을 수시로 조절하고 교육하는 것이 필요합니다.',
   '저녁에 남은 밥의 양으로 반려동물의 식사 습관을 점검할 수 있습니다.'],
  'info': ['반려동물에게 밥을 주었지만 먹지 않는 경우가 있습니다.',
   '특히 아기 강아지들은 밥을 빨리 먹는 것이 건강에 중요합니다.',
   '밥을 잘 먹지 않으면 병원에 가야 할 수도 있습니다.',
   '아기 때는 기력이 매우 중요합니다.',
   '하루에 두 끼 또는 네 끼를 먹는 것이 적절합니다.',
   '하루에 한 끼만 주는 것도 가능합니다.',
   '간식과 밥을 다르게 생각하지 말고, 사료를 간식처럼 맛있게 먹어야 합니다.',
   '밥도 간식처럼 맛있게 먹어야 건강한 상태를 유지할 수 있습니다.',
   '수시로 교육하면서 반려동물의 식사량을 조절해야 합니다.',
   '저녁에 남은 밥의 양을 통해 반려동물의 식사 습관을 파악할 수 있습니다.'],
  'category': '정보',
  'trainer': '강형욱',
  'type': '강아지',
  'breed': '공통'},
 {'id': 1,
  'url': 'https://www.youtube.com/watch?v=5PF5dMzSLZE&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=24&pp=iAQB',
  'main': '강아지 훈련의 기초',
  'sub': 

In [8]:
# 벡터스토어 db 인스턴스를 생성
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7736\630986974.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


In [9]:
from tqdm import tqdm

# 각 문서의 'main'과 'sub' 필드를 합쳐 'content'를 만들고 임베딩
for doc in tqdm(json_data, desc="문서 처리 중", unit="문서"):
    # 'main' 필드 텍스트
    main_text = doc['main']
    
    # 'sub' 필드의 모든 문장을 하나의 문자열로 결합
    sub_text = ' '.join(doc['sub'])
    
    # 'main'과 'sub'를 합쳐 'content' 생성
    content = f"{main_text} {sub_text}"
    
    # 'content' 필드를 문서에 추가
    doc['content'] = content
    
    # 'content'로 임베딩 생성
    embedding = embeddings_model.embed_query(content)
    
    # 문서에 임베딩 추가
    doc['embedding'] = embedding
    
    # 'main'과 'sub' 필드 제거
    del doc['main']
    del doc['sub']

print("'content' 필드가 생성되고 임베딩이 JSON 문서에 추가되었습니다. 'main'과 'sub' 필드가 제거되었습니다.")

# 임베딩의 차원 수 확인 (첫 번째 문서의 임베딩을 사용)
embedding_dim = len(json_data[0]['embedding'])
print(f"임베딩 벡터의 차원 수: {embedding_dim}")

# MongoDB 인덱스 생성을 위한 정보 출력
print("\nMongoDB 인덱스 생성을 위한 정보:")
print("1. 텍스트 인덱스용 필드: 'content'")
print(f"2. 벡터 인덱스용 필드: 'embedding', 차원 수: {embedding_dim}")

문서 처리 중:   5%|▌         | 8/159 [00:06<01:52,  1.35문서/s]

In [ ]:
import json

# JSON 파일로 저장할 경로 지정
output_file_path = 'embedded_data.json'

# JSON 형식으로 데이터 저장
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print(f"임베딩이 포함된 JSON 데이터가 {output_file_path}에 저장되었습니다.")


임베딩이 포함된 JSON 데이터가 embedded_data.json에 저장되었습니다.
